# Proyecto 2 

## Imports

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import Ridge
# from statsmodels.graphics.gofplots import qqplot
# from yellowbrick.regressor import ResidualsPlot
import scipy.stats as stats
# import statsmodels.stats.diagnostic as diag
from scipy.stats import normaltest

## Leer la data y Cleaning

In [54]:
df = pd.read_csv('./data/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Primero, se realiza una descripción del dataset para obtener un resumen estadístico de los datos. Esto nos permite identificar la distribución de las variables, sus valores mínimos y máximos, la media, la mediana y la desviación estándar. Además, nos ayuda a detectar posibles valores atípicos y comprender mejor la escala de los datos antes de realizar cualquier limpieza o transformación.

In [ ]:
df.describe()

Primero, es importante eliminar las filas que contienen datos incompletos. Para ello, contaremos cuántos valores nulos hay en cada columna. Esto nos permitirá determinar si la cantidad de filas con valores faltantes es insignificante. Si encontramos columnas con un número excesivo de valores nulos (según nuestro criterio, más de 40), estas deberán ser eliminadas.

In [55]:
# Encontrar las columnas que tienen los mayores valores nulos
NAValues = [(col, count, (count / len(df)) * 100) for col, count in df.isnull().sum().items() if count > 0]
print(NAValues)

[('LotFrontage', 259, 17.73972602739726), ('Alley', 1369, 93.76712328767123), ('MasVnrType', 872, 59.726027397260275), ('MasVnrArea', 8, 0.547945205479452), ('BsmtQual', 37, 2.5342465753424657), ('BsmtCond', 37, 2.5342465753424657), ('BsmtExposure', 38, 2.6027397260273974), ('BsmtFinType1', 37, 2.5342465753424657), ('BsmtFinType2', 38, 2.6027397260273974), ('Electrical', 1, 0.0684931506849315), ('FireplaceQu', 690, 47.26027397260274), ('GarageType', 81, 5.5479452054794525), ('GarageYrBlt', 81, 5.5479452054794525), ('GarageFinish', 81, 5.5479452054794525), ('GarageQual', 81, 5.5479452054794525), ('GarageCond', 81, 5.5479452054794525), ('PoolQC', 1453, 99.52054794520548), ('Fence', 1179, 80.75342465753424), ('MiscFeature', 1406, 96.30136986301369)]


### Criterios de eliminacion de columnas

Dado que el dataset contiene una gran cantidad de datos, primero realizamos una búsqueda para identificar las columnas con al menos un valor nulo. Posteriormente, analizamos el porcentaje de valores faltantes en cada una de estas columnas. Si el porcentaje de datos nulos era significativo (considerado alto según nuestro criterio), decidimos eliminarlas para evitar sesgos en el análisis y mejorar la calidad de los datos.

### Columas eliminadas
Las siguientes columnas fueron eliminadas debido a su alto porcentaje de valores nulos:

* Alley → 93.77% de datos nulos.
* MasVnrType → 59.73% de datos nulos.
* FireplaceQu → 47.26% de datos nulos.
* GarageQual → 5.55% de datos nulos.
* PoolQC → 99.52% de datos nulos.
* Fence → 80.75% de datos nulos.
* MiscFeature → 96.30% de datos nulos.
* Id -> Es una variable insignificante que no nos aporta nada.

### Posteriormente

Al ya no existir columnas con un algo numero de valores nulos, ya nos podemos deshacer de las filas que estan incompletas.

In [56]:
# Quitar las columnas con mayor porcentaje de nulos
df.drop(['Id', "Alley", "MasVnrType", "FireplaceQu", "GarageQual", "PoolQC", "Fence", "MiscFeature" ], axis=1, inplace=True)
# Ahora ya se pueden quitar las filas con NA porque son muy pocos
df_cleaned = df.dropna()

In [57]:
# Asegurar que no hayan valores nulos
NAValues = list(df_cleaned.isnull().sum() / len(df_cleaned))
list(value for value in NAValues if value > 0)

[]

In [61]:
print("Shape: ",df_cleaned.shape)
print("Duplicados: ",df.duplicated().sum())

Shape:  (1094, 73)
Duplicados:  0
